In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import IsolationForest
import joblib

# ✅ Load dataset
df = pd.read_csv("merged_file.csv", encoding="latin1")

# ✅ Print available columns to check missing features
print("Available Columns:", df.columns.tolist())

# ✅ Convert timestamp columns to datetime format if they exist
for col in df.columns:
    if "date" in col.lower() or "time" in col.lower():
        df[col] = pd.to_datetime(df[col], errors='coerce', dayfirst=True)

# ✅ Define features dynamically based on available columns
TRAINED_FEATURES = [
    "Z-Axis RMS Velocity (in/sec)",
    "Z-Axis RMS Velocity (mm/sec)",
    "Temperature (°F)",
    "Temperature (°C)"
]

FEATURES = [col for col in TRAINED_FEATURES if col in df.columns]

# ✅ Clean numeric columns by removing invalid characters and fixing formatting
def clean_numeric(value):
    if isinstance(value, str):
        value = value.replace(" ", "").replace(",", "").strip()
        value = value.replace("..", ".")  # Fix cases with double dots
        if value.count(".") > 1:  # If multiple dots exist, keep only the first
            parts = value.split(".")
            value = parts[0] + "." + "".join(parts[1:])
        try:
            return float(value)
        except ValueError:
            return np.nan
    return value

df[FEATURES] = df[FEATURES].map(clean_numeric)

# ✅ Handle missing values
df.fillna(df.median(), inplace=True)  # Replace missing values with column medians

# ✅ Detect anomalies and create 'Failure' column
iso_forest = IsolationForest(contamination=0.05, random_state=42)
df["Failure"] = iso_forest.fit_predict(df[FEATURES])
df["Failure"] = df["Failure"].apply(lambda x: 1 if x == -1 else 0)  # Convert anomalies to failure (1)

# ✅ Split dataset
X = df[FEATURES]
y = df["Failure"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ✅ Retrain and save the StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# ✅ Train a predictive maintenance model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_scaled, y_train)

# ✅ Evaluate the model
y_pred = model.predict(X_test_scaled)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

# Save the selected features list
joblib.dump(FEATURES, "selected_features.pkl")

# Save the StandardScaler
joblib.dump(scaler, "scaler2.pkl")

# Save the trained model
joblib.dump(model, "predictive_maintenance_model.pkl")


print("✅ Predictive Maintenance Model Trained and Saved!")


Available Columns: ['Timestamp', 'Z-Axis RMS Velocity (in/sec)', 'Z-Axis RMS Velocity (mm/sec)', 'Temperature (Â°F)', 'Temperature (Â°C)', 'X-Axis RMS Velocity (in/sec)', 'X-Axis RMS Velocity (mm/sec)', 'Z-Axis Peak Acceleration (G)', 'X-Axis Peak Acceleration (G)', 'Z-Axis Peak Velocity (in/sec)', 'Z-Axis Peak Velocity (mm/sec)', 'X-Axis Peak Velocity (in/sec)', 'X-Axis Peak Velocity (mm/sec)', 'Z-Axis High-Frequency RMS Acceleration (G)', 'X-Axis High-Frequency RMS Acceleration (G)']
Accuracy: 1.0
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      9692
           1       1.00      1.00      1.00       486

    accuracy                           1.00     10178
   macro avg       1.00      1.00      1.00     10178
weighted avg       1.00      1.00      1.00     10178

✅ Predictive Maintenance Model Trained and Saved!


In [1]:
import joblib
import numpy as np
from pymodbus.client import ModbusSerialClient
import time

# ✅ Load Model, Scaler, and Features
model = joblib.load("predictive_maintenance_model.pkl")
scaler = joblib.load("scaler2.pkl")
FEATURES = joblib.load("selected_features.pkl")  # Ensure correct feature names

# ✅ Configure Modbus RTU Client
client = ModbusSerialClient(port="COM7", baudrate=19200, stopbits=1, bytesize=8, parity="N", timeout=3)

# ✅ Register Addresses for Only Necessary Features
REGISTER_MAP = {
    "Z-Axis RMS Velocity (in/sec)": 45201 - 40001,
    "Z-Axis RMS Velocity (mm/sec)": 45202 - 40001,
    "Temperature (°F)": 45203 - 40001,
    "Temperature (°C)": 45204 - 40001,
}

SCALING_FACTORS = {
    "Z-Axis RMS Velocity (in/sec)": 6.5535 / 65535,
    "Z-Axis RMS Velocity (mm/sec)": 65.535 / 65535,
    "Temperature (°F)": 327.67 / 32767,
    "Temperature (°C)": 327.67 / 32767,
}

# ✅ Read Modbus Register Function
def read_register(address, scale_factor):
    try:
        response = client.read_holding_registers(address=address, count=1, slave=1)
        if response and response.registers:
            return round(response.registers[0] * scale_factor, 3)
        return None
    except Exception as e:
        print(f"Error reading register {address}: {e}")
        return None

# ✅ Real-time Data Collection and Failure Prediction
def predict_maintenance():
    if client.connect():
        print("✅ Connected to Modbus RTU device")
    else:
        print("❌ Failed to connect")
        return

    while True:
        data = []

        # Collect sensor data
        for param in FEATURES:
            value = read_register(REGISTER_MAP[param], SCALING_FACTORS[param])
            data.append(value)

        import pandas as pd

        # Convert data to DataFrame with correct feature names before scaling
        data_df = pd.DataFrame([data], columns=FEATURES)
        
        # Scale the input data
        data_scaled = scaler.transform(data_df)  # ✅ Fixes warning

        # Predict failure
        failure_prediction = model.predict(data_scaled)[0]
        print(f"🔍 Failure Prediction: {'Failure Detected' if failure_prediction == 1 else 'No Issue'}")

        time.sleep(1)  # Adjust sampling rate

predict_maintenance()
client.close()


✅ Connected to Modbus RTU device
🔍 Failure Prediction: No Issue
🔍 Failure Prediction: No Issue
🔍 Failure Prediction: No Issue
🔍 Failure Prediction: No Issue
🔍 Failure Prediction: No Issue
🔍 Failure Prediction: No Issue
🔍 Failure Prediction: No Issue
🔍 Failure Prediction: No Issue
🔍 Failure Prediction: No Issue
🔍 Failure Prediction: Failure Detected
🔍 Failure Prediction: Failure Detected
🔍 Failure Prediction: Failure Detected
🔍 Failure Prediction: Failure Detected
🔍 Failure Prediction: Failure Detected
🔍 Failure Prediction: Failure Detected
🔍 Failure Prediction: No Issue
🔍 Failure Prediction: No Issue
🔍 Failure Prediction: No Issue
🔍 Failure Prediction: No Issue


KeyboardInterrupt: 